# Sleeper Standings

Use [Sleeper API Documentation](https://docs.sleeper.com/) to complete this.

Sample League Sleeper IDs:
 1. QUESTASY Football Trial: ```865826998382112768```

TODO:
 - [x] Consider using [leg](https://api.sleeper.app/v1/state/nfl) for the "week" variable so you don't have to manually update it weekly
 - [X] Join [Rosters endpoint "owner_id"](https://api.sleeper.app/v1/league/844512859378528256/rosters) with [Users endpoint "user_id"](https://api.sleeper.app/v1/league/844512859378528256/users) to get "display_name" associated with overall stats.
 - [ ] For weekly stats (where week < "leg") use the [matchups endpoint](https://docs.sleeper.com/#getting-matchups-in-a-league) and match user table's roster ID with matchup's roster ID

In [178]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import requests

##
## SETTINGS:
## List of Leagues
## Output: list_leagues (variable; list)
##

leagues = [865826998382112768,\
           #975832259569795072
          ]


#
# Get the current week
#
# variable: num_week
#

url_nfl_state= 'https://api.sleeper.app/v1/state/nfl'
response = requests.get(url_nfl_state)
current_week = response.json()['week']

### TEMPORARY OVERIDE ###
current_week = 13



#
# Join Rosters endpoint "owner_id" with Users endpoint "user_id" to get "display_name" associated with overall stats.
# Input: leagues
# Output table_teams; list_owners
#
# What we need:

# "Display Name": "username", "user_id": 1234, "roster_id": 4, "league_id":512321, "wins":5, "losses":5, ties:0 "points for": 234]

list_rosters_json = []
list_roster_data = []
list_users_data = []
list_matchups_data = []
i = 0

for league in leagues:
    #set display league id (i) to 1 and increment per league
    i+=1
    url_roster = f"https://api.sleeper.app/v1/league/{league}/rosters"
    roster_response = requests.get(url_roster)
    json_roster = roster_response.json()
    # do += instead of append so it keeps it flat
    list_rosters_json+=json_roster
    
#    leagueid, leagueid_display ownerId, rosterId,
    for team in json_roster:
        temp_list = [league, i, team['owner_id'], team['roster_id'], \
                     team['settings']['wins'],team['settings']['losses'],\
                     team['settings']['ties'], \
                     float(team['settings']['fpts']+team['settings']['fpts_decimal']*.01),\
                     float(team['settings']['fpts_against']+team['settings']['fpts_against_decimal']*.01)\
                    ]
        list_roster_data.append(temp_list)

#     get users endpoint
    url_users = f"https://api.sleeper.app/v1/league/{league}/users"
    users_response = requests.get(url_users)
    json_users = users_response.json()
    for team in json_users:
        temp_list = [team['user_id'],team['display_name']]
        list_users_data.append(temp_list)
        
# get matchup endpoint
    for week in range(1,current_week+1):
        url_matchups =f"https://api.sleeper.app/v1/league/{league}/matchups/{week}"
        matchups_response = requests.get(url_matchups)
        json_matchups = matchups_response.json()
        for matchup in json_matchups:
            list_matchups_data.append([league, matchup['roster_id'], matchup['points']])
        

df_roster = pd.DataFrame(list_roster_data, columns =['League ID', 'League', 'Owner ID', 'Roster ID', 'Wins', "Losses", "Ties", "Pts For", "Pts Against"])
display(df_roster)
df_users = pd.DataFrame(list_users_data, columns = ['Owner ID', "Display Name"]) 
display(df_users)




df_matchups = pd.DataFrame(list_matchups_data, columns = ["League ID", "Roster ID", "Pts For"])
df_matchups_2 = df_matchups.copy().drop("Roster ID", axis=1).groupby('League ID').describe()

display(df_matchups_2)
# Do a join?
# https://intellipaat.com/community/19104/python-pandas-how-to-assign-groupby-operation-results-back-to-columns-in-parent-dataframe
# display(df_matchups.groupby('League ID').mean())
# display(df_matchups_2)



df_grouped = df_matchups.groupby(['League ID', 'Roster ID']).describe()
df_grouped_2 = df_grouped.join(df_matchups_2, on="League ID",rsuffix=" League")
df_grouped_2.columns = ["_".join(col_name).rstrip('_') for col_name in df_grouped_2.columns.to_flat_index()]
df_grouped_2 = df_grouped_2.reset_index().drop(["Pts For_count", "Pts For League_count","Pts For League_min","Pts For League_25%","Pts For League_50%","Pts For League_75%","Pts For League_max"],axis = 1)
df_grouped_2 = df_grouped_2.assign(z_score = (df_grouped_2['Pts For_mean'] - df_grouped_2['Pts For League_mean'])/df_grouped_2['Pts For League_std'])
display(df_grouped_2)

df_final = df_roster.join(df_users.set_index('Owner ID'), on='Owner ID', validate='1:1')
df_final = pd.merge(df_final, df_grouped_2, on=['League ID', 'Roster ID'])
df_final = df_final.reindex(columns=["Display Name","League", "Wins", "Losses", "Ties", "Pts For", "Pts Against", "z_score"]).sort_values(by=["Wins","z_score"],ascending=[False,False])
df_final = df_final.reset_index()
df_final.index+=1
df_final = df_final.drop("index",axis=1)
display(df_final)

,League ID,League,Owner ID,Roster ID,Wins,Losses,Ties,Pts For,Pts Against
0,865826998382112768,1,865826147894730752,1,7,6,0,1585.54,1612.72
1,865826998382112768,1,719794343593963520,2,9,4,0,1845.72,1500.62
2,865826998382112768,1,866081206578610176,3,3,10,0,1356.68,1724.02
3,865826998382112768,1,470291496407396352,4,10,3,0,1899.18,1678.96
4,865826998382112768,1,471142445220884480,5,1,12,0,1329.36,1803.68
5,865826998382112768,1,866158791224311808,6,9,4,0,1762.04,1496.96
6,865826998382112768,1,466108799711703040,7,4,9,0,1528.62,1687.96
7,865826998382112768,1,866318462782402560,8,9,4,0,1759.66,1561.88


,Owner ID,Display Name
0,466108799711703040,dustinkang
1,470291496407396352,AlexEJerome
2,471142445220884480,SwagDawgy
3,719794343593963520,sturt
4,865826147894730752,mikeyg7
5,866081206578610176,halleyscomet
6,866158791224311808,Valio
7,866318462782402560,pavanbhat


Pts For                                                \
                     count        mean        std   min      25%     50%   
League ID                                                                  
865826998382112768   104.0  124.603846  32.636486  50.3  102.585  125.99   

                                     
                        75%     max  
League ID                            
865826998382112768  148.445  199.64

,League ID,Roster ID,Pts For_mean,Pts For_std,Pts For_min,Pts For_25%,Pts For_50%,Pts For_75%,Pts For_max,Pts For League_mean,Pts For League_std,z_score
0,865826998382112768,1,120.810769,28.502965,85.66,104.72,115.02,120.66,196.24,124.603846,32.636486,-0.116222
1,865826998382112768,2,141.209231,26.438775,80.64,127.80,152.60,159.40,168.30,124.603846,32.636486,0.508798
2,865826998382112768,3,103.667692,39.115283,50.30,80.44,91.10,124.72,193.64,124.603846,32.636486,-0.641495
3,865826998382112768,4,144.783077,26.867834,104.02,127.26,150.20,159.70,199.64,124.603846,32.636486,0.618303
4,865826998382112768,5,101.027692,37.472457,53.60,70.12,98.58,127.58,191.44,124.603846,32.636486,-0.722386
5,865826998382112768,6,134.541538,18.336954,103.06,119.96,129.60,146.38,162.80,124.603846,32.636486,0.304496
6,865826998382112768,7,116.047692,29.629654,71.94,93.72,112.32,128.18,169.96,124.603846,32.636486,-0.262165
7,865826998382112768,8,134.743077,25.113450,98.68,112.00,132.14,147.86,172.26,124.603846,32.636486,0.310672


,Display Name,League,Wins,Losses,Ties,Pts For,Pts Against,z_score
1,AlexEJerome,1,10,3,0,1899.18,1678.96,0.618303
2,sturt,1,9,4,0,1845.72,1500.62,0.508798
3,pavanbhat,1,9,4,0,1759.66,1561.88,0.310672
4,Valio,1,9,4,0,1762.04,1496.96,0.304496
5,mikeyg7,1,7,6,0,1585.54,1612.72,-0.116222
6,dustinkang,1,4,9,0,1528.62,1687.96,-0.262165
7,halleyscomet,1,3,10,0,1356.68,1724.02,-0.641495
8,SwagDawgy,1,1,12,0,1329.36,1803.68,-0.722386
